In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
import os

def read_json(f):
    f = open(f) 
    return json.load(f) 


def gen_plots(path_dir = "../../results/soi_grad/{}/{}/seed_{}/setting_{}/dim_{}/rho{}.json"
              ,arch="mlp",rho=0.7,bench="mix",seeds=[],setting=0,dim=10,mets=[]):
    gt = {}
    e= {met : []  for met in mets}
    rho = str(rho)
    for i, seed in enumerate( seeds ) :
            file_path = path_dir.format(arch,bench,seed,setting,dim,rho)
            print("doing_file "+ str(file_path))
            out=read_json(file_path)
            for idx, met in enumerate(mets):
                gt[met]= out[rho] ["gt"]["g_o_inf"][idx]
                e[met].append(out[rho] ["e"]["g_o_inf"][met])

    return {"gt":gt, "e": e}


In [ ]:
setting=[0,1]
dims = [5,15]
seeds=[22,11,55,42,33]
arch ="tx"
seeds=[33,55,77]



path = "plots_g/"
if not os.path.exists(path):
    os.makedirs(path)
    
for setting in setting:
    for dim in dims:
        if setting ==0:
            NB = 10
            r = 0.7
        else :
            NB = 6
            r=0.7
         
        data = gen_plots(seeds=seeds,mets=["x"+str(i) for i in range(NB)],
                         setting=setting,rho=r,arch=arch,bench="mix",dim=dim)

        vars =  data["e"].keys()

   
        
        data_plot = { var : data["e"][var] for var in vars }
        df = pd.DataFrame(data_plot)
        df = df.melt(value_vars=vars,var_name=["var"],value_name="val")

        mylabels = [ r'$X^{1}$',r'$X^{2}$',r'$X^{3}$',r'$X^{4}$',r'$X^{5}$',r'$X^{6}$',r'$X^{7}$',r'$X^{8}$',r'$X^{9}$',r'$X^{10}$']
        labels = mylabels[:NB]


        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(4, 3))


        my_pal={"x"+str(i): "steelblue" if  data["gt"]["x"+str(i)]>0 else "tomato" for i in range(NB)}
        
        bplot1  = sns.boxplot(x='var', y='val', data=df, ax=ax1, palette=my_pal,fliersize=0)
        ax1.set_xticklabels(labels)

        plt.scatter(np.arange(NB),[data["gt"]["x"+str(i)] for i in range(NB)], zorder=2,marker='D',s=35,color = 'dimgray',
                    label="GT")
        
        plt.plot(np.arange(NB),0*np.arange(NB), 
                    zorder=-1,color="black",linestyle=":")
        plt.title(r"$\partial \Omega $ for setting {} and dim {}".format(setting,dim),fontsize=14)
        for ax in [ax1]:
            ax.set_xlabel('',)
            ax.set_ylabel(r'$\partial \Omega $',fontsize=14)
            plt.yticks(fontsize=12)
            plt.xticks(fontsize=12)
            ax.legend(fontsize=13)

        plt.savefig("plots_g/grad_setting_{}_dim_{}.png".format(setting,dim),bbox_inches='tight',dpi=300)